# Simulations on c302

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
%pwd

## Functions

**Read Data -** Creates an array from the file containing the data

**Prepare Array -** Changes the array so that it can be represented in a given number of time measures 

**Find Cells -** Finds in the LEMS file the neurons which were used in the simulation

**Change Cell Names -** Converts the cells DB1 to DB01, so they are sorted in the plot. It also sorts the data and changes the columns accordingly to the changes in names. 
(I had an issue with the order of cells. The order was simular to this: VB1, VB10, VB11, VB12, VB2, VB3, VB4 ... and this function changes to VB01, VB02, VB03, VB04...VB11, VB12, but also changes the order of the data from the voltages and calcium dynamics of the neurons)

**Find Muscles -** Finds in the LEMS file the muscles which were used in the simulation

**Plot Data -** Makes a plot using the data given, the legend introduced and the title of the graph

**Plot Three Data -** Makes a three plot using the data given, the legend introduced and the title of the graph

**Plot All Motoneurons -** Function used for the simulations of the ventral nerve cord, it plots in seven different colors and plots, the cells motor neurons from the VNC. 

**Map Plot -** Makes a color array of the data values. It has been used to see the muscular activity of the ismulations.


In [ ]:
def read_data(filename):
    with open(filename) as f:
        line = f.readline()
    col = len(line.split())
    array = np.loadtxt(filename, usecols=range(0,col))
    #array = np.delete(array, 0, 1) #delete first column (time)
    #array = np.delete(array, 0, 0) #delete first row (t=0)
    #array[a,b] = element in row a+1, in column b+1 
    #print(array[0,0])
    return array

In [ ]:
def prepare_array(arr_in, time_measures):
    # The array that comes in: 
    # Has for x the number of neurons/muscles + 1 (one extra column for time values)
    # Has for y the time intervales
    y, x = arr_in.shape
    #print("Number of muscles/neurons: %d" %(x))
    #print("Number of time measures: %d" %(y))
    value = np.floor(y/time_measures)
    value = value.astype(int)
    #print(value)
    arr = np.zeros((time_measures, x))
    #print(arr)
    for i in range(0, time_measures):
        number_to_delete = value - 1
        #print(i*number_to_delete)
        arr[i] = arr_in[i*number_to_delete]
        #print(arr)
        
    
    arr_out = arr
    return arr_out

In [ ]:
def find_cells(LEMS):
    text = open(LEMS, "r")
    count = 0
    s = []
    for j in text:
        s.append(j)
        count = count + 1
    
    def search(word, sentences):
        return [i for i in sentences if re.search(r'\b%s\b' % word, i)]
    strings = search('Cells', s) #Find all the lines in the LEMS file with the word Cells
    
    for i in strings:
        if re.search(r'.*Cells:.*', i):
            cells_str = i
    #print(cells_str) #cells_str is the line with the cells of the simulation of the LEMS file


    parts = re.split('[\']',cells_str)
    #print(len(parts))
    cells = [] #List of cells in the LEMS file
    if len(parts) <3:
        print('No cells in this file')
    else:
        for i in range(len(parts)):
            if (i % 2) != 0:
                cells.append(parts[i])
    #print(cells)
    text.close()
    cells.sort()
    return cells


In [ ]:
def change_cell_names(cells, data_1, data_2):
    #This function changes the names of cells like DA9 to DA09
    #Also, it sorts again the names so that DA09 is bejore DA10
    #Last, it rearranges also the data from the data files so that it moves when the cells are sorted. 
    
    # Change the names of the cells - DA9 to DA09 ... 
    modified_cells = []
    for cell in cells:
        num = [int(char) for char in cell if char.isdigit()]
        if len(num) == 1:
            modified_cells.append(cell[:-1] + '0' + cell[-1:])
        else:
            modified_cells.append(cell)
    cells = modified_cells
    #print(cells)
    
    #Order the data, together with the cells in alphabetical and numerical order
    order = np.argsort(cells)
    cells.sort()

    data_t = np.transpose(data_1)
    data_t_ordered = data_t[0]
    for i in order:
        data_t_ordered = np.vstack((data_t_ordered,data_t[i+1]))
    data_1 = np.transpose(data_t_ordered)
    
    data_t = np.transpose(data_2)
    data_t_ordered = data_t[0]
    for i in order:
        data_t_ordered = np.vstack((data_t_ordered,data_t[i+1]))
    data_2 = np.transpose(data_t_ordered)
    
    return cells, data_1, data_2

In [ ]:
def find_muscles(LEMS):
    text = open(LEMS, "r")
    count = 0
    s = []
    for j in text:
        s.append(j)
        count = count + 1
    
    def search(word, sentences):
        return [i for i in sentences if re.search(r'\b%s\b' % word, i)]
    strings = search('Muscles', s) #Find all the lines in the LEMS file with the word Muscles
    
    for i in strings:
        if re.search(r'.*Muscles:.*', i):
            muscles_str = i
    #print(muscles_str) #muscles_str is the line with the muscles of the simulation of the LEMS file
    
    
    # We need to note that 'Muscles: True' and 'Muscles: All muscles' will print all the muscles!!
    
    if ('All' in muscles_str or 'True' in muscles_str):
        # Create complete muscle List
        muscleList = []
        quadrants = ['DL', 'DR', 'VL', 'VR']
        for i in range(24):
            for quadrant in quadrants:
                #print(quadrant)
                #print(str(i+1))
                if i < 9:
                    muscleList.append('M%s0%s' %(quadrant, str(i+1)))
                else:
                    muscleList.append('M%s%s' %(quadrant, str(i+1)))
        
        muscles = sorted(muscleList)
    else:
        parts = re.split('[\']',muscles_str)
        muscles = []
        if len(parts) < 3:
            print('No muscles in this file')
        else:
            for i in range(len(parts)):
                if (i % 2) != 0:
                    muscles.append(parts[i])
        muscles.sort()
        
    text.close()
    #print(muscles)
    return muscles

In [ ]:
#def plot_data(data, labels, title):
def plot_data(data, labels, title):
    fig = plt.figure(figsize=(12, 6)) #width and heigth
    #'''
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.set_xlabel('time (s)')
    #ax.set(ylim=(-0.075, 0.025))
    #ax.legend(labels)
    data = np.transpose(data)
    #'''
    time = data[0].tolist()
    for i in range(len(labels)):
        value = data[i+1].tolist()
        cell = labels[i]
        ax.plot(time, value, label = cell)
    leg = ax.legend()
    #'''
    

In [ ]:
def plot_three_data(data, labels, title, first_index, second_index, third_index):
    fig = plt.figure(figsize=(12, 6)) #width and heigth
    #'''
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    ax1.set_title(title)
    ax3.set_xlabel('time (s)')
    #ax1.set(ylim=(-0.08, 0.025))
    #ax2.set(ylim=(-0.08, 0.025))
    #ax3.set(ylim=(-0.1, 0))
    #'''
    #ax.legend(labels)
    data = np.transpose(data)

    from cycler import cycler
    #'''
    
    #Colorsets: 
    #DB - [BuGn (0.6, 0.2)]     VB - [YlGn (0.8, 0.4)]
    #DA - [YlOrRd (0.2, 0.5)]     VA -  [Oranges(0.4, 1)]
    #DD - [PuBu (0.2, 0.6)]     VD - [Blues (0.1, 0.5)]
    #AS - [RdPu (0.2, 0.6)]
    #AVA and AVB: 
    #colors =['#cc0000ff', '#6aa84fff']
    
    time = data[0].tolist()
    #colors = [plt.cm.YlOrRd(i) for i in np.linspace(0.2, 0.5, len(first_index))] #YlGn
    colors = ['#a0dacb', '#27613e']
    ax1.set_prop_cycle(cycler('color', colors))
    for i in first_index:
        value = data[i+1].tolist()
        cell = labels[i]
        ax1.plot(time, value, label = cell)
    
    #CODE TO SET LEGEND TO THE RIGHT OUTSIDE
    # Shrink current axis by 20%
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
    
    
    #colors = [plt.cm.Oranges(i) for i in np.linspace(0.4, 1, len(second_index))] #copper
    colors = ['#fec35d', '#793f25']
    ax2.set_prop_cycle(cycler('color', colors))
    for i in second_index:
        value = data[i+1].tolist()
        cell = labels[i]
        ax2.plot(time, value, label = cell)
    leg = ax2.legend(ncol=2)
    
    #Set legent to the right
    box = ax2.get_position()
    ax2.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
    
    
    #colors = [plt.cm.RdPu(i) for i in np.linspace(0.2, 0.6, len(third_index))] #RdBu
    #colors =['#cc0000', '#6aa84f']
    colors = ['#ecbbd5', '#3d78d8','#70aade']
    ax3.set_prop_cycle(cycler('color', colors))
    for i in third_index:
        value = data[i+1].tolist()
        cell = labels[i]
        ax3.plot(time, value, label = cell)
    leg = ax3.legend(ncol=2)
    
    #Set legent to the right
    box = ax3.get_position()
    ax3.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax3.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
    #'''

In [ ]:
def plot_all_motor_neurons(data, labels, title, limits):
    
    # This function plots 7 subplots of all the neuron types in the ventral cord: AS, DA, DB, DD, VD, VB and VA
    
    from cycler import cycler
    range_incl = lambda start, end:range(start, end + 1)
    data = np.transpose(data)
    ######### PLOT FIGUTE #######

    fig = plt.figure(figsize=(12, 12)) #width and heigth
    ax = [fig.add_subplot(num) for num in range_incl(711,717)] #The figute has 7 axes, one for each type of neurons
    ax[0].set_title(title)
    ax[6].set_xlabel('time (s)')

    neuron_names = ['AS', 'DA', 'DB', 'DD', 'VD', 'VB', 'VA']
    i = 0
    time = data[0].tolist()
    cells = labels
    
    for neuron in neuron_names:
        
        ### Set names to plot neurons = AS01, AS02, AS03... AS15
        neurons = [neuron + "{:02d}".format(c) for c in range_incl(1, 15)]
        #print(neurons)
        index = [] #Index will showe where these types of neurons are in the whole array of cells
        for j in range(len(cells)):
            if cells[j] in neurons:
                index.append(j)
        #print(index)


        #Each neuron type is assigned a variety of colors 
        if neuron == 'AS':
            colors = [plt.cm.RdPu(i) for i in np.linspace(0.6, 0.2, len(index))] #AS
        elif neuron == 'DA':
            colors = [plt.cm.YlOrRd(i) for i in np.linspace(0.2, 0.5, len(index))] #DA
        elif neuron == 'DB':
            colors = [plt.cm.BuGn(i) for i in np.linspace(0.6, 0.2, len(index))] #DB
        elif neuron == 'DD':
            colors = [plt.cm.PuBu(i) for i in np.linspace(0.9, 0.5, len(index))] #DD
        elif neuron == 'VD':
            colors = [plt.cm.Blues(i) for i in np.linspace(0.5, 0.3, len(index))] #VD
        elif neuron == 'VB':
            colors = [plt.cm.YlGn(i) for i in np.linspace(0.8, 0.4, len(index))] #VB
        elif neuron == 'VA':
            colors = [plt.cm.Oranges(i) for i in np.linspace(0.4, 1, len(index))] #VA

        ax[i].set_prop_cycle(cycler('color', colors))
        ax[i].set(ylim=limits)
        for j in index:
            value = data[j+1].tolist()
            cell = labels[j]
            ax[i].plot(time, value, label = cell)
        
        #Set the Legend in 2 columns and outside the plot
        leg = ax[i].legend(ncol=2)
        box = ax[i].get_position()
        ax[i].set_position([box.x0, box.y0, box.width * 0.8, box.height])
        ax[i].legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)

        i = i + 1


In [ ]:
def map_plot(arrayName, title):
    fig = plt.figure(figsize=(5, 6)) #width and heigth

    ax = fig.add_subplot(111)
    ax.set_title(title)
    plt.imshow(arrayName).set_cmap('coolwarm')
    ax.set_aspect('auto')
    ax.get_yaxis().set_visible(False)

    cax = fig.add_axes([0.8, 0.1, 0.3, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()

## Main Script 

This script has been used to make all the plots for the Forward and Backward Locomotion model. 

Some examples of what can be done with this code: 

#### Plots of the simple neuron circuits: 

| AB Circuit | Plots |
|:---:|:---:|
| <img src="../images/AB_circuit.png" width="200"> | <img src="../images/simple.png" width="500"> |

#### Plots of all the VNC motor neurons:

FW and BW Model Circuit | Plot 1 | Plot 2
:---: | :---: | :---:
<img src="../images/model_circuit.png" width="200"> | <img src="../images/FWandBW_1.png" width="500"> | <img src="../images/FWandBW_2.png" width="500">

### 1. Reference and parameters of the Simulation
In the next cell you can define the variables that will be used for reading the data.

In [ ]:
reference = 'I_Clamp' #Change the name here
parameters = 'A'
docker_container = 'worm'
include_neurons = True
include_muscles = False
movement = True
isNeuronSimulation = True #Can be done with NEURON or PyNeuroML


### 2. Run the simulation in Docker
This will take a while, make sure you have created a Docker container following the instructions of the README.md

(If GUI applications are not being displayed in yout host, the simulation will not produce data). 

In [ ]:
%cd ..
if isNeuronSimulation: 
    !./run_nrn_c302.sh -n {docker_container}-r {reference} -p {parameters}
else:
    !./run_pynml_c302.sh -n{docker_container} -r {reference} -p {parameters}
%cd c302/


### 3. Read the Data

The following two cells read the data from the simulation and produce the `H_1`, `H_2`, `H_3` and `H_4` arrays with the information of the neuron voltage, neuron activity, muscle voltage and muscle activity, respectively. 

Also, the neurons are ordered into the `cells` variable and muscles into `muscles`. 

In [ ]:
file_start = 'c302_' + parameters + '_' + reference
if isNeuronSimulation:
    path_to_file = 'simulation_data/' + file_start + '_nrn/'
else: 
    path_to_file = 'simulation_data/' + file_start + '/'
neuron_voltage =  path_to_file + file_start + '.dat'
neuron_activity = path_to_file + file_start + '.activity.dat'
muscle_voltage = path_to_file + file_start + '.muscles.dat'
muscle_activity = path_to_file + file_start + '.muscles.activity.dat'
LEMS = path_to_file +'LEMS_c302_' + parameters + '_' + reference + '.xml'
print(neuron_voltage)

In [ ]:
if include_neurons:
    H_1 = prepare_array(read_data(neuron_voltage), 500)
    H_2 = prepare_array(read_data(neuron_activity), 500)
    cells = find_cells(LEMS)
    
    cells, H_1, H_2 = change_cell_names(cells, H_1, H_2)
    
    
    print('Neurons: ')
    print(cells)
    
    

else:
    print ('No neurons in this simulation')
if include_muscles:
    H_3 = prepare_array(read_data(muscle_voltage), 100)
    H_4 = prepare_array(read_data(muscle_activity), 100)
    muscles = find_muscles(LEMS)
    if (len(np.transpose(H_3)) != (len(muscles) + 1)):
        print('ERROR: There is a mismatch in the number of data and muscles')
        print(len(np.transpose(H_3))-1)
        print(len(muscles))
    print('Muscles: ')
    print(muscles)
else: 
    print('No muscles in this simulation')

## Script

In [ ]:
if include_neurons:
    print('Neurons: ')
    print(cells)
    
    data_1 = H_1
    data_2 = H_2
    legend = cells
    
    #################
    # CELLS TO PLOT #
    #################
    
    #'''
    first_cells = ['DB01', 'VB01']
    second_cells = ['VA01', 'DA01']
    third_cells = ['AS01', 'DD01', 'VD01']
    #'''
    
    '''
    range_incl = lambda start, end:range(start, end + 1)
    first_cells = ["DA{:02d}".format(c) for c in range_incl(1, 15)]
    second_cells = ["VA{:02d}".format(c) for c in range_incl(1, 15)]
    #third_cells = ['AVAR', 'AVAL']
    third_cells = ["VD{:02d}".format(c) for c in range_incl(1, 15)]
    #'''
    count = 0
    first_index = []
    second_index = []
    third_index = []
    for i in range(len(cells)):
        if cells[i] in first_cells:
            first_index.append(i)
        elif cells[i] in second_cells:
            second_index.append(i)
        elif cells[i] in third_cells:
            third_index.append(i)
        count = count+1
    
    print(first_index)
    print(second_index)
    print(third_index)
        
    plot_three_data(data_1, legend, 'Neuron Voltage', first_index, second_index, third_index)
    plot_three_data(data_2, legend, 'Neuron Activity', first_index, second_index, third_index)
else:
    print('No neurons to plot')

In [ ]:
plot_all_motor_neurons(H_1, cells, "Neuron Voltages", (-0.12, 0.01))
plot_all_motor_neurons(H_2, cells, "Neuron [Ca2+]", (0, 18e-8))

In [ ]:
array = np.delete(H_3,0,1)
array2 = np.delete(H_4,0,1)


map_plot(np.transpose(array), 'Muscle Voltage')
map_plot(np.transpose(array2), 'Muscle [Ca2+]')




In [ ]:
if include_neurons:
    print('Neurons: ')
    print(cells)
    Q1 = input('Do you want to plot all neurons? [Y/n]')
    if Q1 == 'Y':
        plotAll = True
    elif Q1 == 'n':
        plotAll = False
    else: 
        print('Please respond [Y/n]')
    
    if plotAll:
        data_1 = H_1
        data_2 = H_2
        legend = cells
    else:
        
        transp_data_1 = np.transpose(H_1)[0]
        transp_data_2 = np.transpose(H_2)[0]
        
        insertNeurons = True
        cells2 = []
        while insertNeurons:
            Q2 = input('Insert name of one neuron: (q to quit)')
            if Q2 == 'q':
                insertNeurons = False
            elif Q2 in cells:
                cells2.append(Q2)
            else:
                print('Error: Neuron not found in this simulation!')
        
        cells2 = list(set(cells2)) # Delete possible duplicates
        cells2.sort() # Alphabetical order
        
        for i in range(len(cells)):
            if cells[i] in cells2:
                transp_data_1 = np.vstack((transp_data_1, np.transpose(H_1)[i+1]))
                transp_data_2 = np.vstack((transp_data_2, np.transpose(H_2)[i+1]))
        data_1 = np.transpose(transp_data_1)
        data_2 = np.transpose(transp_data_2)
        legend = cells2
        
    #plot_data(data, legend, title, xlabel, ylabel, ylimits)
    plot_data(data_1, legend, 'Neuron Voltage')
    plot_data(data_2, legend, 'Neuron Activity')
else:
    print('No neurons to plot')

In [ ]:
if include_muscles: 
    H_3 = prepare_array(read_data(muscle_voltage), 100)
    plot_data(H_3)
    H_4 = prepare_array(read_data(muscle_activity), 100)
    plot_data(H_4)
else:
    print('No muscles to plot')

In [ ]:
if movement:
    H_1 = read_data(neuron_voltage)
    H_2 = read_data(neuron_activity)
    H_1 = np.delete(H_1,0,1) #Delete time column
    H_2 = np.delete(H_2,0,1)
    H_1 = prepare_array(H_1, 500)
    H_2 = prepare_array(H_2, 500)
    movement_plot(H_1)
    movement_plot(H_2)
    
else:
    print('No movement plot')

In [ ]:
H_1[0]

In [ ]:
neuron_names = ['AS', 'DA', 'DB', 'DD', 'VB', 'VB', 'VA']
for neuron in neuron_names:
    neurons = [neuron + "{:02d}".format(c) for c in range_incl(1, 15)]
    print(neurons)

In [ ]:
range_incl = lambda start, end:range(start, end + 1)
neuron_names = ['AS', 'DA', 'DB', 'DD', 'VB', 'VB', 'VA']
count = 0
#indexes = np.array[[np.zeros(15)]]
for neuron in neuron_names:
    neurons = [neuron + "{:02d}".format(c) for c in range_incl(1, 15)]
    index = []
    for i in range(len(cells)):
        if cells[i] in neurons:
            index.append(i)
    print(index)
    #indexes = np.vstack((indexes,index))
    count = count + 1